# This notebook is to take apart video feed and turn each frame into an image that then can be run in MegaDetector.

In [ ]:
!pip install opencv-python-headless 

# Keep this just incase an issue happens.


In [ ]:
import os
import cv2
import json
from megadetector.utils import path_utils
from megadetector.visualization import visualization_utils as vis_utils

# Add the directory containing video_utils.py to the system path
import sys
sys.path.append("C:/Users/Public/Documents/MegaDetector_App/MegaDetector/megadetector/detection")

from video_utils import video_folder_to_frames, frames_to_video, get_video_fs, frame_results_to_video_results

# Configure paths
# input_folder = r"C:/Users/Public/Documents/Data/video_data/"
# output_folder_base = r"C:/Users/Public/Documents/Data/video_data/frames/"
# detected_frame_folder_base = r"C:/Users/Public/Documents/Data/video_data_detected/"
# rendered_videos_folder_base = r"C:/Users/Public/Documents/Data/video_data_rendered/"
# json_path = r"C:/Users/Public/Documents/jsons/tests.json"

input_folder = r"C:/Users/Public/Videos/videos_for_MD/ringtail"
output_folder_base = r"C:/Users/Public/Videos/videos_for_MD/ringtail/frames/" # change name to frame folder
detected_frame_folder_base = r"C:/Users/Public/Videos/videos_for_MD/video_data_detected_ringtail/detections" # detections folder change name
rendered_videos_folder_base = r"C:/Users/Public/Videos/videos_for_MD/ringtail/video_data_rendered" # rename 
json_path = r"C:/Users/Public/Documents/jsons/videos_for_MD/RR_ringtail.json"

# Ensure output directories exist
os.makedirs(detected_frame_folder_base, exist_ok=True)
os.makedirs(rendered_videos_folder_base, exist_ok=True)



In [ ]:
# Split videos into frames
frame_filenames_by_video, fs_by_video, video_filenames = video_folder_to_frames(
    input_folder, output_folder_base, recursive=True, n_threads=4, verbose=True
)


In [ ]:
# Runs MegaDetector on the frames
!python "C:/Users/Public/Documents/MegaDetector_App/MegaDetector/megadetector/detection/run_detector_batch.py" \
    MDV5A "{output_folder_base}" "{json_path}" --output_relative_filenames --recursive --checkpoint_frequency 1000 --quiet


In [ ]:
# Post-process results
!python "C:/Users/Public/Documents/MegaDetector_App/MegaDetector/megadetector/postprocessing/postprocess_batch_results.py" \
    "{json_path}" "{detected_frame_folder_base}" --image_base_dir "{output_folder_base}" --num_images_to_sample -1


In [ ]:
# Verify paths and files
print(os.path.isfile(json_path))  # Should print True if the file exists
print(os.path.isdir(output_folder_base))  # Should print True if the directory exists
print(os.path.isdir(detected_frame_folder_base))  # Should print True if the directory exists


In [ ]:
# detected_frames_folder = r"C:/Users/Public/Documents/Data/video_data_detected/detections_animal"
# output_video_filename = r"C:/Users/Public/Documents/Data/video_data_rendered/combined_video.avi" # derp

detected_frames_folder = r"C:/Users/Public/Videos/videos_for_MD/video_data_detected_ringtail/detections"
output_video_filename = r"C:/Users/Public/Videos/RR_ringtail.AVI" # make sure to have .mp4 or .avi or it will not work

# Ensure output directory exists
os.makedirs(os.path.dirname(output_video_filename), exist_ok=True)


# Collect all frame files from the detected frames folder
frame_files = []
for root, dirs, files in os.walk(detected_frames_folder):
    for file in sorted(files):
        if file.endswith('.jpg') or file.endswith('.png'):
            frame_files.append(os.path.join(root, file))

# Get frame rate from one of the original videos
# Assuming all original videos have the same frame rate; you can modify this as needed
# sample_video_file = 'C:/Users/Public/Documents/Data/video_data/2835528-hd_1920_1080_25fps.mp4'
# this is saving the frame rate to the video you want to generate with the same frame rate as this sample video
sample_video_file = "C:/Users/Public/WLF_Cam_Data/videos_for_MD/ringtail/IMG_0528.AVI" # file path of the orginal video, must have file extension
Fs = get_video_fs(sample_video_file)

# Combine all frames into a single video
if frame_files:
    # Determine the width and height from the first image
    frame = cv2.imread(frame_files[0])
    height, width, channels = frame.shape

    # Define the codec and create VideoWriter object
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 files
    output_video_filename = output_video_filename.replace('.avi', '.mp4')
    out = cv2.VideoWriter(output_video_filename, fourcc, Fs, (width, height))

    for image in frame_files:
        frame = cv2.imread(image)
        out.write(frame)

    out.release()

    print(f"Combined video created: {output_video_filename}")
else:
    print("No frames found to combine into a video.")



